Gabriel Meira Delgado

NUSP 9269027

# Resumo

Construi o código que se resume em uma classe Grafo a qual depende fortemente do vetor de adjacência (vetor de listas encadeadas) e da lista de nomes dos vértices (para saber a quantidade de vértices e criar o vetor de adjacência). O grafo é construído através da leitura dos arquivos txt de lista de nome e matriz de adjacência, e está automatizado até certo ponto (se mudarmos o tamanho da lista e matriz mas mantermos a estrutura dos arquivos ele aceitará).

Todos os métodos da classe estão funcionando corretamente e foram deenvolvidos por mim baseado na lógica vista em aula (explicada e/ou nos slides). Cada método está comentado com o item do arquivo de exercício que ele responde ou está associado.

OBS: Não consegui utilizar a notação global para criar as variáveis do DSF como globais, então as criei como atributo da classe Grafo.

# 0. Classe vetor de listas encadeadas

In [1]:
class Registro:
  def __init__(self,dado):
    self.dado = dado
    self.prox = None

class VetorRegistros:
  def __init__(self,N):
    self._tamanho = N
    self.v = [None]*N

  def add_final(self,index,elem):
    reg = Registro(elem)
    if self.v[index] == None: #vértice ainda não tem aresta
      self.v[index] = reg #inserção aresta
    else: #se o vértice já tem aresta
      aux = self.v[index]
      while aux.prox: #ate encontrar um ponteiro vazio
        aux = aux.prox
      aux.prox = reg #inserção aresta
    return

class Fila:
  def __init__(self):
    self.primeiro = None
    self._tamanho = 0
  
  def inserir(self,p):
    if self.primeiro == None:
      self.primeiro = Registro(p)
    else:
      aux = self.primeiro
      while aux.prox:
        aux = aux.prox
      aux.prox = Registro(p)
    self._tamanho += 1

  def __len__(self):
    return self._tamanho

  def remover(self):
    aux = self.primeiro.dado
    self.primeiro = self.primeiro.prox
    self._tamanho -= 1
    return aux

# 1. Classe Grafo

In [8]:
class Grafo:
  def __init__(self):
    aux = open(r'/content/drive/MyDrive/entrada/names_g.txt','r')
    aux = aux.readlines()
    N = len(aux)
    for i in range(N-1):
      aux[i] = aux[i][1:len(aux[i])-2]
    aux[i+1] = aux[i+1][1:len(aux[i+1])-1] #a última linha não tem \n
    self.ListaNomes = aux
    self.VetAdj = VetorRegistros(N)
    M = open(r'/content/drive/MyDrive/entrada/matriz_g.txt','r')
    M = M.readlines()
    for i in range(N):
      for j in range(N):
        if M[i][2*j] == '1': #compensando espaços entre os dados nas colunas
          self.VetAdj.add_final(i,j)
    self.t = 0 #tempo DSF
    self.t1 = [None]*N #tempo de inicialização DSF
    self.t2 = [None]*N #tempo de fechamento DSF
    self.p = [None]*N #pai DSF
    self.color = ['white']*N #cor DSF
    self.grupos = []

  ### 1.a) Achar vértices adjacentess ###
  def AdjDeV(self,nome): #retorna nome dos vertices adjacentes
    p = 0
    for n in self.ListaNomes: #encontra posição p do nome passado
      if n == nome:
        break
      p += 1
    aux = self.VetAdj.v[p]
    print('Os vértices adjacentes de '+nome+' são: \n')
    while aux:
      print(self.ListaNomes[aux.dado])
      aux = aux.prox
    return

  def _ConjAdj(self,index,transposta=False): #retorna lista com nro dos vertices adjacentes
    if transposta:
      N = len(self.ListaNomes)
      conj = []
      for i in range(N): #criar conj dos adjacentes de u no grafo transposto
        if self.VetAdj.v[i]:
          aux = self.VetAdj.v[i]
          while aux.dado <= index:
            if aux.dado == index:
              conj.append(i)
              break
            elif aux.prox:
              aux = aux.prox
            else:
              break
        else:
          continue
      return conj
    else:
      aux = self.VetAdj.v[index]
      conj = []
      while aux:
        conj.append(aux.dado)
        aux = aux.prox
      return conj
  
  ### 1.b) Calcular grau de saída do vértice ###
  def GrauSaida(self,nome):
    N = len(self.ListaNomes)
    p = 0
    for n in self.ListaNomes: #encontra posição p do nome passado
      if n == nome:
        break
      p += 1
    if p == N:
      raise IndexError('Vértice não encontrado')
    aux = self.VetAdj.v[p]
    g = 0
    while aux:
      aux = aux.prox
      g += 1
    return print('o grau de',nome,'saída é: ', g)

  ### 1.c) Encontrar menor caminho entre 2 vértices usando BSF ###
  def MenorCaminho(self,nomeFonte,nomeObjetivo): #Beseado na lógica dos slides
    N = len(self.ListaNomes)
    f = 0
    for n in self.ListaNomes: #encontra posição f da fonte
      if n == nomeFonte:
        break
      f += 1
    if f == N:
      raise IndexError('Vértice fonte não encontrado')
    o = 0
    for n in self.ListaNomes: #encontra posição o do objetivo
      if n == nomeObjetivo:
        break
      o += 1
    if o == N:
      raise IndexError('Vértice objetivo não encontrado')
    color = ['white']*65
    p = [None]*N #nro do vértice pai
    d = [None]*N #diatância da fonte f
    color[f] = 'gray'
    p[f] = []
    d[f] = 0
    Q = Fila()
    Q.inserir(f)
    while len(Q) != 0:
      u = Q.remover()
      for v in self._ConjAdj(u):
        if color[v] == 'white':
          color[v] = 'gray'
          d[v] = d[u]+1
          p[v] = u
          Q.inserir(v)
      color[u] = 'black'
      if u == o:
        break
    print('A menor distância entre ',nomeFonte,' e ',nomeObjetivo,' é: ',d[o])
    ## Mostrar caminho ##
    pai = p[o]
    caminho = [self.ListaNomes[o]]
    while pai != []:
      caminho.append(self.ListaNomes[pai])
      pai = p[pai]
    print('Pelo menos um dos menores caminhos é: ')
    for vertice in reversed(caminho):
      print(vertice)

  def Visitar(self,u,transposta=False):
    N = len(self.ListaNomes)
    if transposta: #visita ao grafo transposto
      self.color[u] = 'gray'
      for v in self._ConjAdj(u,transposta):
        if self.color[v] == 'white':
          self.p[v] = u
          # print(self.ListaNomes[v])
          self.grupos[self.t].append(self.ListaNomes[v])
          self.Visitar(v,transposta)
      self.color[u] = 'black'  
    else: #visita normal
      self.color[u] = 'gray'
      self.t += 1
      self.t1[u] = self.t
      for v in self._ConjAdj(u):
        if self.color[v] == 'white':
          self.p[v] = u
          self.Visitar(v)
      self.color[u] = 'black'  
      self.t += 1
      self.t2[u] = self.t  

  def BuscarProfundidade(self):
    N = len(self.ListaNomes)
    self.color = ['white']*N
    self.p = [None]*N #nro do vértice pai
    self.t1 = [None]*N #tempo de inicialização
    self.t2 = [None]*N #tempo de fechamento
    self.t = 0
    for i in range(N):
      if self.color[i] == 'white':
        self.Visitar(i)

  ### 1.d) Encontrar componentes fortemente conectados usando DSF ###
  def BuscarCompFortementeConect(self): #baseado na lógica dos slides
    N = len(self.ListaNomes)
    self.BuscarProfundidade()
    self.color = ['white']*N
    self.p = [None]*N #nro do vértice pai
    ordem = []
    for i in range(N):
      aux = self.t2.index(max(self.t2))
      ordem.append(aux)
      self.t2[aux] = 0
    self.grupos = []
    self.t = 0
    for i in ordem:
      if self.color[i] == 'white':
        self.grupos.append([self.ListaNomes[i]])
        self.Visitar(i,True)
        self.t += 1
    ## print dos grupos ##
    print('Existem',self.t,'grupos de componentes fortemente conectados \n')
    for i in range(self.t):
      aux = self.grupos[i]
      aux.sort()
      print('Grupo ',i+1,'\n')
      print(*aux,sep='\n')
      print('\n\n')

  def RemoverVertice(self,nomeVertice): #remove todas arestas ligadas ao vértice passado
    N = len(self.ListaNomes)
    i = 0
    for n in self.ListaNomes: #encontra o nro do vertice
      if n == nomeVertice:
        break
      i += 1
    if i == N:
      raise IndexError('Vértice não encontrado')
    self.VetAdj.v[i] = None #apaga arestas que saem do vértice
    for j in range(N):
      if self.VetAdj.v[j]:
        aux = self.VetAdj.v[j]
        while aux.dado <= i:
          if aux.dado == i and aux == self.VetAdj.v[j]:
            self.VetAdj.v[j] = aux.prox
            break
          elif aux.dado == i:
            aux_anterior.prox = aux.prox
            break
          elif aux.prox:
            aux_anterior = aux
            aux = aux.prox
          else:
            break
        
    

# 2.a) Impressão do grau de saída e vérticess adjacentes

In [9]:
G = Grafo()
G.AdjDeV('pC')
G.GrauSaida('pC')


Os vértices adjacentes de pC são: 

Reaction11
Reaction13
Reaction23
Reaction24
Reaction42
o grau de pC saída é:  5


# 2.b) Impressão do menor caminho

In [4]:
G.MenorCaminho('pA','pC')

A menor distância entre  pA  e  pC  é:  14
Pelo menos um dos menores caminhos é: 
pA
Reaction15
pA_OpB1
Reaction16
OpB
Reaction3
pol_OpB
Reaction4
pol
Reaction5
pol_OpC
Reaction33
mC
Reaction39
pC


# 2.c) Impressão componentes fortemente conectados

In [5]:
G.BuscarCompFortementeConect()

Existem 8 grupos de componentes fortemente conectados 

Grupo  1 

OpA
OpB
OpC
Reaction1
Reaction10
Reaction11
Reaction12
Reaction13
Reaction14
Reaction15
Reaction16
Reaction17
Reaction18
Reaction19
Reaction2
Reaction20
Reaction21
Reaction22
Reaction23
Reaction24
Reaction25
Reaction26
Reaction27
Reaction28
Reaction29
Reaction3
Reaction30
Reaction31
Reaction32
Reaction33
Reaction37
Reaction38
Reaction39
Reaction4
Reaction5
Reaction6
Reaction7
Reaction8
Reaction9
mA
mB
mC
pA
pA_OpB1
pA_OpB2
pA_OpB_pA
pB
pB_OpC1
pB_OpC2
pB_OpC_pB
pC
pC_OpA1
pC_OpA2
pC_OpA_pC
pol
pol_OpA
pol_OpB
pol_OpC



Grupo  2 

Reaction40



Grupo  3 

Reaction35



Grupo  4 

Reaction36



Grupo  5 

Reaction41



Grupo  6 

Reaction34



Grupo  7 

Reaction42



Grupo  8 

trash





# 3.a) Análise da remoção de 'Reaction2'

In [6]:
G.RemoverVertice('Reaction2')
G.BuscarCompFortementeConect()

Existem 9 grupos de componentes fortemente conectados 

Grupo  1 

Reaction2



Grupo  2 

OpA
OpB
OpC
Reaction1
Reaction10
Reaction11
Reaction12
Reaction13
Reaction14
Reaction15
Reaction16
Reaction17
Reaction18
Reaction19
Reaction20
Reaction21
Reaction22
Reaction23
Reaction24
Reaction25
Reaction26
Reaction27
Reaction28
Reaction29
Reaction3
Reaction30
Reaction31
Reaction32
Reaction33
Reaction37
Reaction38
Reaction39
Reaction4
Reaction5
Reaction6
Reaction7
Reaction8
Reaction9
mA
mB
mC
pA
pA_OpB1
pA_OpB2
pA_OpB_pA
pB
pB_OpC1
pB_OpC2
pB_OpC_pB
pC
pC_OpA1
pC_OpA2
pC_OpA_pC
pol
pol_OpA
pol_OpB
pol_OpC



Grupo  3 

Reaction40



Grupo  4 

Reaction35



Grupo  5 

Reaction36



Grupo  6 

Reaction41



Grupo  7 

Reaction42



Grupo  8 

Reaction34



Grupo  9 

trash





# 3.b) Análise da remoção 'Reaction1'

In [7]:
G = Grafo()
G.RemoverVertice('Reaction1')
G.BuscarCompFortementeConect()

Existem 15 grupos de componentes fortemente conectados 

Grupo  1 

Reaction1



Grupo  2 

pol_OpA



Grupo  3 

Reaction31



Grupo  4 

Reaction2



Grupo  5 

Reaction37
mA



Grupo  6 

Reaction34



Grupo  7 

OpB
OpC
Reaction10
Reaction15
Reaction16
Reaction17
Reaction18
Reaction19
Reaction20
Reaction21
Reaction22
Reaction27
Reaction28
Reaction29
Reaction3
Reaction30
Reaction32
Reaction33
Reaction38
Reaction4
Reaction5
Reaction6
Reaction7
Reaction8
Reaction9
mB
pA
pA_OpB1
pA_OpB2
pA_OpB_pA
pB
pB_OpC1
pB_OpC2
pB_OpC_pB
pol
pol_OpB
pol_OpC



Grupo  8 

Reaction40



Grupo  9 

Reaction35



Grupo  10 

Reaction39
mC



Grupo  11 

OpA
Reaction11
Reaction12
Reaction13
Reaction14
Reaction23
Reaction24
Reaction25
Reaction26
pC
pC_OpA1
pC_OpA2
pC_OpA_pC



Grupo  12 

Reaction42



Grupo  13 

Reaction36



Grupo  14 

Reaction41



Grupo  15 

trash





# Conclusão

A remoção da Reaction2 não afeta o ciclo principal pois os vértices pA e pC continuam sendo fortemente conectados (membros do Grupo 2 no item 3.a)). No entanto, a remoção da Reaction1 afeta o ciclo principal pois agora os vértices pA e pC não são mais fortemente conectados (pA pertence ao Grupo 7 e pC ao grupo 11 no item 3.b)).

Gerar PDF

In [ ]:
% cd drive/MyDrive/Colab\ Notebooks
! sudo apt update
! sudo apt-get install texlive-full

!jupyter nbconvert --to pdf Trabalho2_grafos.ipynb